# Стекинг и Блендинг

In [1]:
import catboost

In [2]:
from catboost import CatBoostClassifier

In [3]:
CatBoostClassifier()

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Загрузка
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

### Загрузка датасета

In [3]:
# Для блендинга нужен большой датасет, поэтому давайте создадим свой!
data, target = datasets.make_classification(n_samples=100000, n_features = 15, n_informative = 12, n_classes = 2, 
                                            n_redundant = 0, n_clusters_per_class = 1, 
                                            random_state = 7)

In [4]:
# Положим данные в табличку
df = pd.DataFrame(data=data, index=range(len(data)), columns=range(len(data[0])))
t_df = pd.DataFrame(data=target)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.032784,0.121863,1.152660,-1.824953,0.695797,3.623146,-1.427768,-0.907597,-0.259047,-2.617477,-1.162208,-0.942514,0.361030,0.047440,-2.414601
1,0.003259,-0.266702,-0.172534,0.317077,-1.272421,-0.500436,-5.172817,2.400920,-3.226955,5.236402,-1.392474,1.453011,0.651480,-0.405567,-0.047236
2,-3.361852,0.328164,-0.676021,-2.679977,1.209057,2.391206,-1.455813,2.559608,0.715355,-0.741438,-0.497015,-4.828820,-0.250569,0.026710,-0.377829
3,-3.023783,-2.627421,0.290723,-6.082231,0.757969,-1.261731,-1.137153,5.710433,0.380537,0.073292,1.782437,3.590135,0.498531,1.949923,-2.412880
4,-3.043163,-0.712612,1.113411,-3.008166,0.201240,3.097968,0.377683,0.964651,0.680228,-2.986106,0.952898,-2.510878,1.341273,1.772567,-3.179428


In [6]:
# Посмотрим на баланс классов нашей выборки
t_df[0].value_counts()

1    50007
0    49993
Name: 0, dtype: int64

### Сформируем отложенную выборку, на которой будем оценивать результаты

In [7]:
# Разделите данные на трейн и тест
X_df, X_test, y_df, y_test = train_test_split(df, t_df, random_state=322, test_size=0.2)
# Разделите трейн выборку на 2 части для блендинга
X_train, X_valid, y_train, y_valid = train_test_split(X_df, y_df, random_state=322, test_size=0.4)

In [8]:
# Тест-выборка
print(X_test.shape, y_test.shape)
# Выборка для обучения базовых моделей
print(X_train.shape, y_train.shape)
# Выборка для обучения мета-алгоритма
print(X_valid.shape, y_valid.shape)

(20000, 15) (20000, 1)
(48000, 15) (48000, 1)
(32000, 15) (32000, 1)


### Обучаем базовые алгоритмы

Почему мы используем линейную регрессию в задаче классификации?

Дело в том, что при создании метапризнаков более важную роль играет не те модели, которые создают метапризнаки, а сам факт создания метафичей. Кроме того, в презентации говорилось, что лучше использовать модели с разной породой.

Однако не надо бездумно стекать модели! Сейчас, цель упражнения понять основную идею метода. В будущем подходите к выбору модели для стекинга и блендинга вдумчиво:)

In [9]:
# Создаем базовые модели, можете поиграться с параметрами
log = LogisticRegression()
lin = LinearRegression()
rf = RandomForestClassifier()

In [11]:
# Обучаем базовые модели на первой части обучающей выборки
log.fit(X_train,y_train)
lin.fit(X_train,y_train)
rf.fit(X_train,y_train)

RandomForestClassifier()

In [12]:
# создаем метапризнаки с помощью предобученных базовых моделей на оставшейся части трейна
# по сути это предсказания моделями таргета
meta_pred_log = log.predict_proba(X_valid)
meta_pred_lin = lin.predict(X_valid)
meta_pred_rf = rf.predict(X_valid)

In [13]:
# Формируем минидатасет из метапризнаков
meta_df = pd.DataFrame(data=0, index=range(len(X_valid)), columns=['log', 'lin', 'rf'])

meta_df['log'] = meta_pred_log
meta_df['lin'] = meta_pred_lin
meta_df['rf'] = meta_pred_rf

In [14]:
# Посмотрите, что получилось
meta_df

,log,lin,rf
0,0.979010,0.129101,0
1,0.008624,0.991087,1
2,0.007930,1.008216,1
3,0.304501,0.591517,1
4,0.022485,0.890615,1
...,...,...,...
31995,0.812118,0.374930,0
31996,0.998591,-0.095267,0
31997,0.004034,1.072925,1
31998,0.577507,0.480139,0


In [15]:
# Создайте метапризнаки для теста
meta_test_pred_log = log.predict_proba(X_test)
meta_test_pred_lin = lin.predict(X_test)
meta_test_pred_rf = rf.predict(X_test)

In [16]:
# Сформируйте из них датасет
meta_test_df = pd.DataFrame(data=0, index=range(len(X_test)), columns=['log', 'lin', 'rf'])

meta_test_df['log'] = meta_test_pred_log
meta_test_df['lin'] = meta_test_pred_lin
meta_test_df['rf'] = meta_test_pred_rf

In [17]:
# Посмотрите, что получилось
meta_test_df

,log,lin,rf
0,0.698326,0.430785,0
1,0.117174,0.714137,1
2,0.999892,-0.358075,0
3,0.013876,0.946337,1
4,0.991858,0.064901,0
...,...,...,...
19995,0.723047,0.426283,0
19996,0.703470,0.443887,0
19997,0.115811,0.727287,1
19998,0.093751,0.741558,1


### Обучаем алгоритм и метаалгоритм

In [18]:
# Создаем метаалгоритмы
metaalg = LogisticRegression()
alg = LogisticRegression()

# Обучаем их
metaalg.fit(meta_df, y_valid) # обучаем на признаках из meta_df

alg.fit(X_df, y_df) # обучаем на признаках из X_df

LogisticRegression()

In [18]:
# Сделайте предсказания для X_test метаалгоритмом и обычным алгоритмом
meta_y_pred = metaalg.predict(meta_test_df)
y_pred = alg.predict(X_test)

### Сравниваем качество

In [19]:
# Посмотрим ошибку
print('Ошибка блендинга: ', mean_squared_error(y_test, meta_y_pred))
print('Ошибка модели: ', mean_squared_error(y_test, y_pred))

Ошибка блендинга:  0.0396
Ошибка модели:  0.09645
